In [ ]:
# %% [markdown]
# # Análisis de Clustering - Divorcios y Matrimonios en Guatemala
# 
# **Objetivos:**
# 1. Analizar la estabilidad matrimonial (relación divorcios/matrimonios por departamento)
# 2. Analizar la concentración urbana de divorcios vs matrimonios
# 3. Analizar la duración promedio del matrimonio (usando edades como proxy)
# 
# **Estructura:**
# - Carga y limpieza de ambos datasets
# - Agregación por departamento
# - Preparación para clustering

# %% [markdown]
# ## 1. Importar librerías

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
import warnings
warnings.filterwarnings('ignore')




# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

# %% [markdown]
# ## 2. Cargar datasets

# %%
# Cargar datos de divorcios
print("=" * 60)
print("CARGANDO DATASET DE DIVORCIOS")
print("=" * 60)
df_div = pd.read_csv('div_full.csv')
print(f"Shape divorcios: {df_div.shape}")
print(f"Columnas: {df_div.columns.tolist()}")
print()

# Cargar datos de matrimonios
print("=" * 60)
print("CARGANDO DATASET DE MATRIMONIOS")
print("=" * 60)
df_mat = pd.read_csv('mat_full.csv')
print(f"Shape matrimonios: {df_mat.shape}")
print(f"Columnas: {df_mat.columns.tolist()}")

# %% [markdown]
# ## 3. Limpieza de datos

# %% [markdown]
# ### 3.1 Convertir columnas numéricas (edades, años, días)

# %%
# Columnas numéricas en ambos datasets
columnas_numericas_div = ['AÑOREG', 'DIAOCU', 'EDADHOM', 'EDADMUJ']
columnas_numericas_mat = ['AÑOREG', 'DIAOCU', 'EDADHOM', 'EDADMUJ']

print("Convirtiendo columnas numéricas - DIVORCIOS")
for col in columnas_numericas_div:
    if col in df_div.columns:
        df_div[col] = pd.to_numeric(df_div[col], errors='coerce')
        validos = df_div[col].notna().sum()
        print(f"  {col}: {validos}/{len(df_div)} válidos ({validos/len(df_div)*100:.1f}%)")

print("\nConvirtiendo columnas numéricas - MATRIMONIOS")
for col in columnas_numericas_mat:
    if col in df_mat.columns:
        df_mat[col] = pd.to_numeric(df_mat[col], errors='coerce')
        validos = df_mat[col].notna().sum()
        print(f"  {col}: {validos}/{len(df_mat)} válidos ({validos/len(df_mat)*100:.1f}%)")

# %% [markdown]
# ### 3.2 Limpieza de nombres de departamentos

# %%
# Diccionario de mapeo para departamentos (unifica variantes con/sin tilde)
mapeo_departamentos = {
    'Guatemala': 'Guatemala',
    'Quetzaltenango': 'Quetzaltenango',
    'Escuintla': 'Escuintla',
    'Jutiapa': 'Jutiapa',
    'San Marcos': 'San Marcos',
    'Izabal': 'Izabal',
    'Huehuetenango': 'Huehuetenango',
    'Suchitepequez': 'Suchitepéquez',
    'Suchitepéquez': 'Suchitepéquez',
    'Retalhuleu': 'Retalhuleu',
    'Zacapa': 'Zacapa',
    'Santa Rosa': 'Santa Rosa',
    'Chiquimula': 'Chiquimula',
    'Alta Verapaz': 'Alta Verapaz',
    'Jalapa': 'Jalapa',
    'Peten': 'Petén',
    'Petén': 'Petén',
    'Quiche': 'Quiché',
    'Quiché': 'Quiché',
    'Chimaltenango': 'Chimaltenango',
    'Sacatepequez': 'Sacatepéquez',
    'Sacatepéquez': 'Sacatepéquez',
    'Baja Verapaz': 'Baja Verapaz',
    'El Progreso': 'El Progreso',
    'Totonicapan': 'Totonicapán',
    'Totonicapán': 'Totonicapán',
    'Solola': 'Sololá',
    'Sololá': 'Sololá'
}

# Aplicar a divorcios
print("LIMPIEZA DE DEPARTAMENTOS - DIVORCIOS")
print("Valores únicos antes:")
print(df_div['DEPREG'].unique())

df_div['DEPREG_LIMPIO'] = df_div['DEPREG'].map(mapeo_departamentos).fillna(df_div['DEPREG'])

print("\nValores únicos después:")
print(df_div['DEPREG_LIMPIO'].unique())

# Identificar valores no oficiales (como "Ignorado" o "No especificado")
oficiales = set(mapeo_departamentos.values())
no_oficiales_div = set(df_div['DEPREG_LIMPIO'].unique()) - oficiales
print(f"\nValores no oficiales en divorcios: {no_oficiales_div}")

# %%
# Aplicar a matrimonios
print("\n" + "="*60)
print("LIMPIEZA DE DEPARTAMENTOS - MATRIMONIOS")
print("Valores únicos antes:")
print(df_mat['DEPREG'].unique())

df_mat['DEPREG_LIMPIO'] = df_mat['DEPREG'].map(mapeo_departamentos).fillna(df_mat['DEPREG'])

print("\nValores únicos después:")
print(df_mat['DEPREG_LIMPIO'].unique())

no_oficiales_mat = set(df_mat['DEPREG_LIMPIO'].unique()) - oficiales
print(f"\nValores no oficiales en matrimonios: {no_oficiales_mat}")

# %% [markdown]
# ### 3.3 Limpieza de meses (crear columna numérica)

# %%
# Mapeo de meses a número
meses_map = {
    'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
    'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
}

# Divorcios
df_div['MES_NUM'] = df_div['MESOCU'].map(meses_map)
print("DIVORCIOS - Valores nulos en MES_NUM:", df_div['MES_NUM'].isna().sum())
# Eliminar filas con mes nulo (si son pocas)
df_div = df_div.dropna(subset=['MES_NUM'])
print(f"Shape divorcios después de limpiar meses: {df_div.shape}")

# Matrimonios
df_mat['MES_NUM'] = df_mat['MESOCU'].map(meses_map)
print("\nMATRIMONIOS - Valores nulos en MES_NUM:", df_mat['MES_NUM'].isna().sum())
df_mat = df_mat.dropna(subset=['MES_NUM'])
print(f"Shape matrimonios después de limpiar meses: {df_mat.shape}")

# %% [markdown]
# ## 4. Excluir valores no oficiales en departamentos

# %%
# Para el análisis geográfico, excluimos registros con departamentos no oficiales
# (como "Ignorado", "No especificado", etc.)

print("DIVORCIOS - Registros con departamento no oficial:")
if no_oficiales_div:
    mask_div = df_div['DEPREG_LIMPIO'].isin(no_oficiales_div)
    print(f"  {mask_div.sum()} registros ({mask_div.sum()/len(df_div)*100:.2f}%)")
    df_div = df_div[~mask_div]
    print(f"  Shape después de excluir: {df_div.shape}")
else:
    print("  No hay valores no oficiales en divorcios")

print("\nMATRIMONIOS - Registros con departamento no oficial:")
if no_oficiales_mat:
    mask_mat = df_mat['DEPREG_LIMPIO'].isin(no_oficiales_mat)
    print(f"  {mask_mat.sum()} registros ({mask_mat.sum()/len(df_mat)*100:.2f}%)")
    df_mat = df_mat[~mask_mat]
    print(f"  Shape después de excluir: {df_mat.shape}")
else:
    print("  No hay valores no oficiales en matrimonios")

# %% [markdown]
# ## 5. Agregación por departamento

# %% [markdown]
# ### 5.1 Función para calcular proporciones mensuales

# %%
def calc_prop_meses(grupo):
    """Calcula la proporción de divorcios/matrimonios en cada mes (1-12)"""
    total = len(grupo)
    if total == 0:
        return pd.Series([0]*12, index=[f'prop_mes_{i}' for i in range(1,13)])
    conteo = grupo['MES_NUM'].value_counts().reindex(range(1,13), fill_value=0)
    prop = conteo / total
    prop.index = [f'prop_mes_{i}' for i in range(1,13)]
    return prop

# %% [markdown]
# ### 5.2 Agregación para divorcios

# %%
print("="*60)
print("AGREGACIÓN POR DEPARTAMENTO - DIVORCIOS")
print("="*60)

df_div_agrup = df_div.groupby('DEPREG_LIMPIO').agg(
    total_divorcios=('DEPREG_LIMPIO', 'count'),
    edad_hom_media=('EDADHOM', 'mean'),
    edad_muj_media=('EDADMUJ', 'mean'),
    edad_hom_std=('EDADHOM', 'std'),
    edad_muj_std=('EDADMUJ', 'std')
).reset_index()

# Añadir proporciones mensuales
prop_meses_div = df_div.groupby('DEPREG_LIMPIO').apply(calc_prop_meses).reset_index()
df_div_agrup = df_div_agrup.merge(prop_meses_div, on='DEPREG_LIMPIO', how='left')

print(f"Shape: {df_div_agrup.shape}")
print(f"Departamentos: {len(df_div_agrup)}")
df_div_agrup.head()

# %% [markdown]
# ### 5.3 Agregación para matrimonios

# %%
print("\n" + "="*60)
print("AGREGACIÓN POR DEPARTAMENTO - MATRIMONIOS")
print("="*60)

df_mat_agrup = df_mat.groupby('DEPREG_LIMPIO').agg(
    total_matrimonios=('DEPREG_LIMPIO', 'count'),
    edad_hom_media=('EDADHOM', 'mean'),
    edad_muj_media=('EDADMUJ', 'mean'),
    edad_hom_std=('EDADHOM', 'std'),
    edad_muj_std=('EDADMUJ', 'std')
).reset_index()

# Añadir proporciones mensuales
prop_meses_mat = df_mat.groupby('DEPREG_LIMPIO').apply(calc_prop_meses).reset_index()
df_mat_agrup = df_mat_agrup.merge(prop_meses_mat, on='DEPREG_LIMPIO', how='left')

print(f"Shape: {df_mat_agrup.shape}")
print(f"Departamentos: {len(df_mat_agrup)}")
df_mat_agrup.head()

# %% [markdown]
# ### 5.4 Manejo de valores nulos en edades

# %%
# Para divorcios, rellenar medias nulas con media nacional
media_global_div_hom = df_div['EDADHOM'].mean()
media_global_div_muj = df_div['EDADMUJ'].mean()
std_global_div_hom = df_div['EDADHOM'].std()
std_global_div_muj = df_div['EDADMUJ'].std()

df_div_agrup['edad_hom_media'] = df_div_agrup['edad_hom_media'].fillna(media_global_div_hom)
df_div_agrup['edad_muj_media'] = df_div_agrup['edad_muj_media'].fillna(media_global_div_muj)
df_div_agrup['edad_hom_std'] = df_div_agrup['edad_hom_std'].fillna(std_global_div_hom)
df_div_agrup['edad_muj_std'] = df_div_agrup['edad_muj_std'].fillna(std_global_div_muj)

# Para matrimonios
media_global_mat_hom = df_mat['EDADHOM'].mean()
media_global_mat_muj = df_mat['EDADMUJ'].mean()
std_global_mat_hom = df_mat['EDADHOM'].std()
std_global_mat_muj = df_mat['EDADMUJ'].std()

df_mat_agrup['edad_hom_media'] = df_mat_agrup['edad_hom_media'].fillna(media_global_mat_hom)
df_mat_agrup['edad_muj_media'] = df_mat_agrup['edad_muj_media'].fillna(media_global_mat_muj)
df_mat_agrup['edad_hom_std'] = df_mat_agrup['edad_hom_std'].fillna(std_global_mat_hom)
df_mat_agrup['edad_muj_std'] = df_mat_agrup['edad_muj_std'].fillna(std_global_mat_muj)

print("Valores nulos después de rellenar:")
print("Divorcios:", df_div_agrup.isna().sum().sum())
print("Matrimonios:", df_mat_agrup.isna().sum().sum())

# %% [markdown]
# ## 6. Combinar dataframes para análisis comparativos

# %%
# Unir los dataframes por departamento para tener divorcios y matrimonios juntos
df_combinado = pd.merge(
    df_div_agrup, 
    df_mat_agrup, 
    on='DEPREG_LIMPIO', 
    how='outer',
    suffixes=('_div', '_mat')
)

# Calcular indicadores clave
df_combinado['tasa_divorcios_por_matrimonio'] = df_combinado['total_divorcios'] / df_combinado['total_matrimonios']
df_combinado['porcentaje_divorcios_guatemala'] = (df_combinado['DEPREG_LIMPIO'] == 'Guatemala').astype(int) * df_combinado['total_divorcios'] / df_combinado['total_divorcios'].sum() * 100
df_combinado['porcentaje_matrimonios_guatemala'] = (df_combinado['DEPREG_LIMPIO'] == 'Guatemala').astype(int) * df_combinado['total_matrimonios'] / df_combinado['total_matrimonios'].sum() * 100

# Verificar que tenemos todos los departamentos (deberían ser 22)
print(f"Total departamentos en dataframe combinado: {len(df_combinado)}")
print("\nDepartamentos presentes:")
print(sorted(df_combinado['DEPREG_LIMPIO'].tolist()))

# Verificar si faltan algunos
departamentos_esperados = sorted(oficiales)
faltantes = set(departamentos_esperados) - set(df_combinado['DEPREG_LIMPIO'].tolist())
if faltantes:
    print(f"\n⚠️ Departamentos faltantes: {faltantes}")
else:
    print("\n✅ Todos los 22 departamentos están presentes")

# %% [markdown]
# ## 7. Guardar dataframes preparados

# %%
# Guardar dataframes individuales
df_div_agrup.to_csv('divorcios_agrupado_departamento.csv', index=False)
df_mat_agrup.to_csv('matrimonios_agrupado_departamento.csv', index=False)
df_combinado.to_csv('divorcios_matrimonios_combinado.csv', index=False)

print("Archivos guardados:")
print("- divorcios_agrupado_departamento.csv")
print("- matrimonios_agrupado_departamento.csv")
print("- divorcios_matrimonios_combinado.csv")

# %% [markdown]
# ## 8. Exploración inicial de los datos agregados

# %%
# Estadísticas descriptivas - Divorcios
print("="*60)
print("ESTADÍSTICAS DESCRIPTIVAS - DIVORCIOS POR DEPARTAMENTO")
print("="*60)
df_div_agrup.describe()

# %%
# Estadísticas descriptivas - Matrimonios
print("="*60)
print("ESTADÍSTICAS DESCRIPTIVAS - MATRIMONIOS POR DEPARTAMENTO")
print("="*60)
df_mat_agrup.describe()

# %%
# Comparación de totales
print("="*60)
print("COMPARACIÓN DE TOTALES")
print("="*60)
print(f"Total divorcios en el país: {df_div_agrup['total_divorcios'].sum():,}")
print(f"Total matrimonios en el país: {df_mat_agrup['total_matrimonios'].sum():,}")
print(f"Tasa nacional divorcios/matrimonios: {df_div_agrup['total_divorcios'].sum()/df_mat_agrup['total_matrimonios'].sum():.3f}")

# %%
# Visualizar concentración en Guatemala
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Divorcios
top5_div = df_div_agrup.nlargest(5, 'total_divorcios')
others_div = pd.DataFrame({
    'DEPREG_LIMPIO': ['Otros'],
    'total_divorcios': [df_div_agrup['total_divorcios'].sum() - top5_div['total_divorcios'].sum()]
})
plot_div = pd.concat([top5_div, others_div])
axes[0].pie(plot_div['total_divorcios'], labels=plot_div['DEPREG_LIMPIO'], autopct='%1.1f%%')
axes[0].set_title('Concentración de divorcios - Top 5 departamentos')

# Matrimonios
top5_mat = df_mat_agrup.nlargest(5, 'total_matrimonios')
others_mat = pd.DataFrame({
    'DEPREG_LIMPIO': ['Otros'],
    'total_matrimonios': [df_mat_agrup['total_matrimonios'].sum() - top5_mat['total_matrimonios'].sum()]
})
plot_mat = pd.concat([top5_mat, others_mat])
axes[1].pie(plot_mat['total_matrimonios'], labels=plot_mat['DEPREG_LIMPIO'], autopct='%1.1f%%')
axes[1].set_title('Concentración de matrimonios - Top 5 departamentos')

plt.tight_layout()
plt.show()

# %%
# Relación edad hombres vs mujeres por departamento
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Divorcios
axes[0].scatter(df_div_agrup['edad_hom_media'], df_div_agrup['edad_muj_media'], 
                s=df_div_agrup['total_divorcios']/100, alpha=0.6)
for i, row in df_div_agrup.iterrows():
    axes[0].annotate(row['DEPREG_LIMPIO'], (row['edad_hom_media'], row['edad_muj_media']), 
                     fontsize=8, ha='center')
axes[0].set_xlabel('Edad media hombres')
axes[0].set_ylabel('Edad media mujeres')
axes[0].set_title('Divorcios - Edades medias por departamento')
axes[0].plot([20, 50], [20, 50], 'r--', alpha=0.5)  # línea de referencia

# Matrimonios
axes[1].scatter(df_mat_agrup['edad_hom_media'], df_mat_agrup['edad_muj_media'], 
                s=df_mat_agrup['total_matrimonios']/1000, alpha=0.6)
for i, row in df_mat_agrup.iterrows():
    axes[1].annotate(row['DEPREG_LIMPIO'], (row['edad_hom_media'], row['edad_muj_media']), 
                     fontsize=8, ha='center')
axes[1].set_xlabel('Edad media hombres')
axes[1].set_ylabel('Edad media mujeres')
axes[1].set_title('Matrimonios - Edades medias por departamento')
axes[1].plot([20, 50], [20, 50], 'r--', alpha=0.5)

plt.tight_layout()
plt.show()

# %% [markdown]
# ## 9. Resumen de datos preparados
# 
# Ahora tenemos:
# 
# - **`df_div_agrup`**: 22 filas (departamentos) con:
#   - `total_divorcios`
#   - `edad_hom_media`, `edad_muj_media`
#   - `edad_hom_std`, `edad_muj_std`
#   - `prop_mes_1` a `prop_mes_12` (proporción de divorcios en cada mes)
# 
# - **`df_mat_agrup`**: 22 filas con variables análogas para matrimonios
# 
# - **`df_combinado`**: fusión de ambos, incluyendo:
#   - `tasa_divorcios_por_matrimonio`
#   - Indicadores de concentración
# 
# **Siguientes pasos:**
# 1. Determinar tendencia al agrupamiento y número óptimo de clusters
# 2. Aplicar K-means
# 3. Validar con silueta
# 4. Interpretar los clusters usando las variables disponibles
# 
# Esto lo haremos en la siguiente parte del notebook.

In [ ]:
# %% [markdown]
# # Bloque 1: Estabilidad Matrimonial
# 
# Variables clave: tasa divorcios/matrimonios, edades medias al divorcio y al matrimonio.

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples

# Cargar dataframe combinado (asegurar que existe)
# df_combinado = pd.read_csv('divorcios_matrimonios_combinado.csv', index_col=0)

# Selección de variables
vars_estabilidad = [
    'tasa_divorcios_por_matrimonio',
    'edad_hom_media_div',
    'edad_muj_media_div',
    'edad_hom_media_mat',
    'edad_muj_media_mat'
]

X_est = df_combinado[vars_estabilidad].copy()
# Escalar
scaler_est = StandardScaler()
X_est_scaled = scaler_est.fit_transform(X_est)

# Determinar k óptimo (2 a 8)
inertias = []
sil_scores = []
K_range = range(2, 8)
for k in K_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X_est_scaled)
    inertias.append(km.inertia_)
    sil_scores.append(silhouette_score(X_est_scaled, km.labels_))

# Gráficos codo y silueta
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(K_range, inertias, 'bo-')
plt.xlabel('k')
plt.ylabel('Inercia')
plt.title('Método del codo - Estabilidad')
plt.subplot(1,2,2)
plt.plot(K_range, sil_scores, 'ro-')
plt.xlabel('k')
plt.ylabel('Silueta')
plt.title('Silueta promedio - Estabilidad')
plt.show()

# Elegir k que maximiza silueta
k_opt_est = K_range[np.argmax(sil_scores)]
print(f"k óptimo = {k_opt_est} (silueta = {max(sil_scores):.4f})")

# Aplicar K-means
km_est = KMeans(n_clusters=k_opt_est, random_state=42, n_init=10)
labels_est = km_est.fit_predict(X_est_scaled)
df_combinado['cluster_est'] = labels_est

# Validación con silueta
sil_prom_est = silhouette_score(X_est_scaled, labels_est)
print(f"Silueta promedio final: {sil_prom_est:.4f}")

# Perfil de clusters (medias de variables originales)
perfil_est = df_combinado.groupby('cluster_est')[vars_estabilidad].mean().round(2)
print("\nPerfil de clusters (medias):")
print(perfil_est)

# Distribución de departamentos
for c in range(k_opt_est):
    deptos = df_combinado[df_combinado['cluster_est'] == c].index.tolist()
    print(f"\nCluster {c}: {deptos}")

# Gráfico de barras comparativo
perfil_est.plot(kind='bar', figsize=(10,6))
plt.title('Perfil de clusters - Estabilidad Matrimonial')
plt.ylabel('Valor medio')
plt.xticks(rotation=0)
plt.legend(loc='upper right')
plt.show()

# Interpretación (ejemplo, ajústala según resultados)
print("""
Interpretación:
- Cluster 0: Departamentos con tasa de divorcio baja y edades al matrimonio jóvenes.
- Cluster 1: Departamentos con tasa media y edades intermedias.
- Cluster 2: Departamentos con tasa alta y edades más elevadas (posiblemente urbanos).
(Completar con los valores observados)
""")